In [ ]:
import streamlit as st
st.image("Snowflake_Logo.svg", width=300)

# Honor Health Intelligence Agent - ML Models

**Training 3 Machine Learning Models for SDOH and Value-Based Care**

This notebook trains 3 ML models for the Honor Health Intelligence Agent:
1. **READMISSION_RISK_PREDICTOR** - Predicts 30-day readmission risk (2 classes)
2. **HEALTH_OUTCOME_PREDICTOR** - Predicts health outcome improvement (3 classes)
3. **SOCIAL_RISK_STRATIFICATION** - Detects patients by social risk (3 classes)

---

## Prerequisites
- Database: `HONORHEALTH_INTELLIGENCE`
- Schema: `ML_MODELS`
- Feature views created (V_READMISSION_RISK_FEATURES, V_HEALTH_OUTCOME_PREDICTION_FEATURES, V_SOCIAL_RISK_STRATIFICATION_FEATURES)
- Packages: `snowflake-ml-python`, `scikit-learn`, `pandas`

In [ ]:
import os
print(os.listdir('.'))  # Lists all files in current directory

## Setup and Imports

In [ ]:
# Import required libraries
from snowflake.snowpark import Session
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.linear_model import LogisticRegression
from snowflake.ml.modeling.preprocessing import OneHotEncoder, StandardScaler
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.registry import Registry
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

In [ ]:
# Get current session
session = Session.builder.getOrCreate()

# Set context
session.use_database("HONORHEALTH_INTELLIGENCE")
session.use_schema("ML_MODELS")
session.use_warehouse("HONORHEALTH_WH")

print("✅ Session configured")
print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")
print(f"Warehouse: {session.get_current_warehouse()}")

In [ ]:
# Initialize Model Registry
registry = Registry(
    session=session,
    database_name="HONORHEALTH_INTELLIGENCE",
    schema_name="ML_MODELS"
)

print("✅ Model Registry initialized")

---
## Model 1: Loan Default Risk Predictor

**Objective**: Predict likelihood of loan default  
**Labels**: 0=Low Risk, 1=Medium Risk, 2=High Risk, 3=Critical Risk  
**Algorithm**: Random Forest Classifier  
**Features**: Loan amount, term, interest rate, credit score, DTI, LTV, payment history

In [ ]:
# Load loan default feature data
default_df = session.table("HONORHEALTH_INTELLIGENCE.ANALYTICS.V_READMISSION_RISK_FEATURES")

print(f"✅ Loaded {default_df.count()} records for readmission prediction")
default_df.show(5)

In [ ]:
# Split data for training and testing
train_default, test_default = default_df.random_split([0.8, 0.2], seed=42)

# Drop ID columns not needed for training
train_default = train_default.drop("ENCOUNTER_ID")
test_default = test_default.drop("ENCOUNTER_ID")

print(f"Training set: {train_default.count()} records")
print(f"Test set: {test_default.count()} records")

In [ ]:
# Create FAST readmission prediction pipeline - optimized for <10s execution
# Using simpler model: fewer trees, shallow depth, no scaling
default_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["LOAN_TYPE"],
        output_cols=["LOAN_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["READMISSION_LABEL"],
        output_cols=["PREDICTED_READMISSION"],
        n_estimators=3,
        max_depth=3,
        random_state=42
    ))
])

print("✅ Default prediction pipeline created (optimized for speed)")

In [ ]:
# Train the readmission prediction model
print("Training readmission prediction model...")
default_pipeline.fit(train_default)
print("✅ Default prediction model trained")

In [ ]:
# Evaluate model on test set
test_predictions = default_pipeline.predict(test_default)
test_results = test_predictions.select("READMISSION_LABEL", "PREDICTED_READMISSION").to_pandas()

from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(test_results['READMISSION_LABEL'], test_results['PREDICTED_READMISSION'])

print(f"Test Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(
    test_results['READMISSION_LABEL'], 
    test_results['PREDICTED_READMISSION']
))

In [ ]:
# Register model in Model Registry
# Drop label column from sample data - model signature should only include features
sample_data = train_default.drop("READMISSION_LABEL").limit(100)

registry.log_model(
    model=default_pipeline,
    model_name="READMISSION_RISK_PREDICTOR",
    target_platforms=['WAREHOUSE'],
    sample_input_data=sample_data,
    comment="Predicts loan default risk with 4 risk levels (Low/Medium/High/Critical)"
)

print("✅ READMISSION_RISK_PREDICTOR registered in Model Registry")

---
## Model 2: Loan Approval Predictor

**Objective**: Predict loan application approval likelihood  
**Labels**: 0=Likely Deny, 1=Needs Review, 2=Likely Approve  
**Algorithm**: Logistic Regression  
**Features**: Credit score, income, DTI, employment, LTV, collateral

In [ ]:
# Load loan approval feature data
approval_df = session.table("HONORHEALTH_INTELLIGENCE.ANALYTICS.V_HEALTH_OUTCOME_PREDICTION_FEATURES")

print(f"✅ Loaded {approval_df.count()} records for health outcome prediction")
approval_df.show(5)

In [ ]:
# Split data
train_approval, test_approval = approval_df.random_split([0.8, 0.2], seed=42)

train_approval = train_approval.drop("OUTCOME_ID")
test_approval = test_approval.drop("OUTCOME_ID")

print(f"Training set: {train_approval.count()} records")
print(f"Test set: {test_approval.count()} records")

In [ ]:
# Create FAST health outcome prediction pipeline - optimized for <10s execution
# Using LogisticRegression with fewer iterations, no scaling
approval_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["LOAN_TYPE"],
        output_cols=["LOAN_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", LogisticRegression(
        label_cols=["OUTCOME_LABEL"],
        output_cols=["PREDICTED_OUTCOME"],
        max_iter=100
    ))
])

print("✅ Approval prediction pipeline created (optimized for speed)")

In [ ]:
# Train the health outcome prediction model
print("Training health outcome prediction model...")
approval_pipeline.fit(train_approval)
print("✅ Approval prediction model trained")

In [ ]:
# Evaluate model
test_predictions = approval_pipeline.predict(test_approval)
test_results = test_predictions.select("OUTCOME_LABEL", "PREDICTED_OUTCOME").to_pandas()

accuracy = accuracy_score(test_results['OUTCOME_LABEL'], test_results['PREDICTED_OUTCOME'])

print(f"Test Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(
    test_results['OUTCOME_LABEL'], 
    test_results['PREDICTED_OUTCOME'],
    target_names=['Likely Deny', 'Needs Review', 'Likely Approve']
))

In [ ]:
# Register model
# Drop label column from sample data - model signature should only include features
sample_data = train_approval.drop("OUTCOME_LABEL").limit(100)

registry.log_model(
    model=approval_pipeline,
    model_name="HEALTH_OUTCOME_PREDICTOR",
    target_platforms=['WAREHOUSE'],
    sample_input_data=sample_data,
    comment="Predicts loan approval likelihood with 3 outcomes (Deny/Review/Approve)"
)

print("✅ HEALTH_OUTCOME_PREDICTOR registered in Model Registry")

---
## Model 3: Fraud Detection Model

**Objective**: Detect potentially fraudulent loan applications  
**Labels**: 0=Clean, 1=Suspicious, 2=High Risk  
**Algorithm**: Random Forest Classifier  
**Features**: Application velocity, income verification, credit anomalies, DTI flags

In [ ]:
# Load social risk stratification feature data
fraud_df = session.table("HONORHEALTH_INTELLIGENCE.ANALYTICS.V_SOCIAL_RISK_STRATIFICATION_FEATURES")

print(f"✅ Loaded {fraud_df.count()} records for social risk stratification")
fraud_df.show(5)

In [ ]:
# Split data
train_fraud, test_fraud = fraud_df.random_split([0.8, 0.2], seed=42)

train_fraud = train_fraud.drop("OUTCOME_ID")
test_fraud = test_fraud.drop("OUTCOME_ID")

print(f"Training set: {train_fraud.count()} records")
print(f"Test set: {test_fraud.count()} records")

In [ ]:
# Create FAST social risk stratification pipeline - optimized for <10s execution
# Using simpler RandomForest: fewer trees, shallow depth, no scaling
fraud_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["LOAN_TYPE"],
        output_cols=["LOAN_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["RISK_LEVEL_LABEL"],
        output_cols=["PREDICTED_RISK_LEVEL"],
        n_estimators=3,
        max_depth=3,
        random_state=42
    ))
])

print("✅ Fraud detection pipeline created (optimized for speed)")

In [ ]:
# Train the social risk stratification model
print("Training social risk stratification model...")
fraud_pipeline.fit(train_fraud)
print("✅ Fraud detection model trained")

In [ ]:
# Evaluate model
test_predictions = fraud_pipeline.predict(test_fraud)
test_results = test_predictions.select("RISK_LEVEL_LABEL", "PREDICTED_RISK_LEVEL").to_pandas()

accuracy = accuracy_score(test_results['RISK_LEVEL_LABEL'], test_results['PREDICTED_RISK_LEVEL'])

print(f"Test Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(
    test_results['RISK_LEVEL_LABEL'], 
    test_results['PREDICTED_RISK_LEVEL'],
    target_names=['Clean', 'Suspicious', 'High Risk']
))

In [ ]:
# Register model
# Drop label column from sample data - model signature should only include features
sample_data = train_fraud.drop("RISK_LEVEL_LABEL").limit(100)

registry.log_model(
    model=fraud_pipeline,
    model_name="SOCIAL_RISK_STRATIFICATION",
    target_platforms=['WAREHOUSE'],
    sample_input_data=sample_data,
    comment="Detects fraudulent applications with 3 risk levels (Clean/Suspicious/High Risk)"
)

print("✅ SOCIAL_RISK_STRATIFICATION registered in Model Registry")

---
## Summary and Verification

In [ ]:
# List all registered models
models = session.sql("SHOW MODELS IN SCHEMA ML_MODELS").collect()

print("\n" + "="*80)
print("REGISTERED MODELS")
print("="*80)
for model in models:
    print(f"✅ {model['name']}")

print("\n" + "="*80)
print("MODEL TRAINING COMPLETE")
print("="*80)
print("\n3 ML models successfully trained and registered:")
print("1. READMISSION_RISK_PREDICTOR - Predicts default risk (4 classes)")
print("2. HEALTH_OUTCOME_PREDICTOR - Predicts approval likelihood (3 classes)")
print("3. SOCIAL_RISK_STRATIFICATION - Detects fraud risk (3 classes)")
print("\nNext steps:")
print("1. Run honorhealth_07_ml_model_functions.sql to create SQL procedures")
print("2. Run honorhealth_08_intelligence_agent.sql to configure agent")
print("3. Test agent with sample questions from origence_questions.md")